# JSON Schema generation

A simple usage of the library that, given generates a JSON Schema for inputs and outputs.

## 1. Parsing

In this sample we'll show the access from a remote public URL.

In [45]:
from cwl2ogc import load_converter_from_location

cwl_converter = load_converter_from_location('https://raw.githubusercontent.com/eoap/cwl2ogc/refs/heads/develop/tests/artifacts/cwl-types/array-inputs.cwl')

# 2. Inputs JSON Schema generation

Once the document is parsed, invoke the `cwl2ogc` APIs to convert the CWL inputs to the JSON schema:

In [46]:
import sys

cwl_converter.dump_inputs_json_schema(stream=sys.stdout, pretty_print=True)

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "$id": "https://eoap.github.io/cwl2ogc/array-inputs/inputs.yaml",
  "description": "The schema to represent a array-inputs inputs definition",
  "type": "object",
  "required": [
    "filesA",
    "filesB",
    "filesC"
  ],
  "properties": {
    "filesA": {
      "$ref": "#/$defs/filesA"
    },
    "filesB": {
      "$ref": "#/$defs/filesB"
    },
    "filesC": {
      "$ref": "#/$defs/filesC"
    }
  },
  "$defs": {
    "filesA": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "filesB": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "filesC": {
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  }
}

## 2.1 Inputs validation

Schema can be used to fully validate an inputs dictionary (expecting JSON Schema validation errors in the example below):

In [47]:
from jsonschema import Draft202012Validator
from jsonschema.exceptions import (
    SchemaError,
    ValidationError
)

inputs = {
    "aoi": "-118.985,38.432,-118.183,38.938",
    "filesB": "EPSG:4326",
    "bands": [ "green", "nir08" ],
    "item": {
        "class":"https://raw.githubusercontent.com/eoap/schemas/main/url.yaml#URL",
        "value":"https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2/items/LC08_L2SP_042033_20231007_02_T1"
    }
}

try:
    validator = Draft202012Validator(cwl_converter.get_inputs_json_schema())

    errors = validator.iter_errors(inputs)
    if errors:
        for error in validator.iter_errors(inputs):
            print(f"* {'.'.join(error.schema_path)}: {'.'.join(error.path)} - {error.message}")
except SchemaError as schema_error:
    print(schema_error.message)

* required:  - 'filesA' is a required property
* required:  - 'filesC' is a required property
* properties.filesB.type: filesB - 'EPSG:4326' is not of type 'array'


# 3. Outputs JSON Schema generation

Users can reuse the `BaseCWLtypes2OGCConverter` instance to convert the CWL outputs to the JSON Schema:

In [48]:
cwl_converter.dump_outputs_json_schema(stream=sys.stdout, pretty_print=True)

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "$id": "https://eoap.github.io/cwl2ogc/array-inputs/outputs.yaml",
  "description": "The schema to represent a array-inputs outputs definition",
  "type": "object",
  "required": [
    "example_out"
  ],
  "properties": {
    "example_out": {
      "$ref": "#/$defs/example_out"
    }
  },
  "$defs": {
    "example_out": {
      "type": "string"
    }
  }
}